# Determine KLM Operators
### Author: Sarah
### Reviewer: Claudia

## Structured Analysis
### Import all relevant libraries

In [5]:
!pip3 install pingouin

In [6]:
import pandas as pd
import pingouin as pg
import seaborn as sns
from matplotlib import pyplot as plt

## K-Operator
In order to calculate the time for pressing a key, one key (e.g. 8) is pressed ten times in succession.
This is repaeted ten times.
The timestamp is logged, when a key is pressed.

### Read csv file

In [7]:
# raw_data_k_operator = pd.read_csv("./k_operator.csv", sep=';', engine='python')

### Calculate time for one K
For calculation the first timestamp is ignored, because you need time for orientation.
The time difference between the last and the second timestamp is calculated.
The result is divided by the number of keystrokes (equal to the length of the dataframe) minus one to get the time for one keystroke. 

## B-Operator
To calculate the button click, the same button is pressed ten times in a row. 
This is repeated for ten times.
In this case, it was logged when a button is clicked (press + relaese).

### Read csv file

In [8]:
#raw_data_b_operator = pd.read_csv("./b_operator.csv", sep=';', engine='python')

### Calculate time for one B
The first timestamp is ignored as before.
The time difference between the last and the second timestamp is calculated.
The result is divided by the number of button press and relaese (equal to the length of the dataframe) minus one.
To get the time for one button press or release the result is divided by two. 

## P-Operator
To calculate the pointing of the mouse, the mouse pointer is moved to each button in arbitrary order. 
This is repeated ten times.
Each time a button is reached the timestamp is logged.

### Read csv file

In [9]:
#raw_data_p_operator = pd.read_csv("./p_operator.csv", sep=';', engine='python')

### Calculate time for one P
The first timestamp is ignored as before.
The time difference between the last and the second timestamp is calculated and divided by the number of movements plus one to get the time for one P.

## H-Operator
To calculate the time for switching between keyboard and mouse, ???
This is repeated 10 times.

### Read csv file

### Calculate time for one H
The first timestamp is ignored as before.
The time difference between the last and the second timestamp is calculated.
